In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_html_code(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Ошибка при получении HTML-кода: {response.status_code}")
            return None
    except Exception as e:
        print(f"Произошла ошибка при получении HTML-кода: {e}")
        return None

def parse_news_data(html_code):

    soup = BeautifulSoup(html_code, "html.parser")

    news_data = []
    news_items = soup.find_all(class_ ='item news_item item--covered')
    for item in news_items:
        date = item.find(class_ ='item_date-day').text
        month = item.find(class_ ='item_date-month').text
        title = item.find(class_ ='item_text').text
        link = item.find(class_ ='item_link')["href"]
        news_data.append({
            "date": date,
            "month": month,
            "title": title,
            "link": "https://www.company.rt.ru"+link
        })

    news_items = soup.find_all(class_ ='item news_item')
    for item in news_items:
        date = item.find(class_ ='item_date-day').text
        month = item.find(class_ ='item_date-month').text
        title = item.find(class_ ='item_text').text
        link = item.find(class_ ='item_link')["href"]
        news_data.append({
            "date": date,
            "month": month,
            "title": title,
            "link": "https://www.company.rt.ru"+link
        })

    return news_data


url = "https://www.company.rt.ru/ir/news_calendar/"
url2 = "https://www.company.rt.ru/ir/news_calendar/2024/07-06-05-04-03-02-01/"
html_code = get_html_code(url)
html_code2 = get_html_code(url2)
news_data = parse_news_data(html_code) + parse_news_data(html_code2)


df = pd.DataFrame(news_data)
df

,date,month,title,link
0,16,Декабря,Вектор — на устойчивое развитие: «Ростелеком» ...,https://www.company.rt.ru/ir/news_calendar/d47...
1,13,Ноября,Дмитрий Медведев провел заседание совета дирек...,https://www.company.rt.ru/ir/news_calendar/d47...
2,12,Ноября,Финансовые и операционные результаты деятельно...,https://www.company.rt.ru/ir/news_calendar/d47...
3,30,Октября,AK&M подтвердило высший рейтинг нефинансовой о...,https://www.company.rt.ru/ir/news_calendar/d47...
4,02,Октября,Дмитрий Медведев избран председателем Совета д...,https://www.company.rt.ru/ir/news_calendar/d47...
5,13,Ноября,«Ростелеком» разместил биржевые облигации на с...,https://www.company.rt.ru/ir/news_calendar/d47...
6,16,Сентября,«Ростелеком» сообщает об итогах годового общег...,https://www.company.rt.ru/ir/news_calendar/d47...
7,11,Сентября,«Ростелеком» разместил биржевые облигации на с...,https://www.company.rt.ru/ir/news_calendar/d47...
8,14,Августа,Финансовые и операционные результаты деятельно...,https://www.company.rt.ru/ir/news_calendar/d47...
9,13,Августа,Совет директоров «Ростелекома» принял решение ...,https://www.company.rt.ru/ir/news_calendar/d47...


In [5]:
import sqlite3

# Создание базы данных
conn = sqlite3.connect('news.db')

cursor = conn.cursor()
cursor.execute('''
 CREATE TABLE IF NOT EXISTS news (
 id INTEGER PRIMARY KEY,
 date TEXT,
 month TEXT,
 title TEXT,
 link TEXT
 )
''')
conn.commit()

# Закрытие соединения с базой данных
conn.close()

In [7]:
import sqlite3

# Установление соединения с базой данных
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

# Пример словаря с данными
#news_data

# Преобразование словаря в SQL-запрос для вставки данных
columns = ', '.join('?' * len(news_data[0]))
placeholders = ', '.join(['?'] * len(news_data[0]))
sql = f'INSERT INTO news ({", ".join(news_data[0].keys())}) VALUES ({placeholders})'

# Выполнение SQL-запроса
cursor.executemany(sql, [tuple(item.values()) for item in news_data])

# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

In [9]:
import sqlite3

# Установление соединения с базой данных
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

# Выполнение SQL-запроса для получения данных из таблицы
cursor.execute("SELECT * FROM news")
rows = cursor.fetchall()

# Вывод данных на экран
for row in rows:
 print(row)

# Закрытие соединения с базой данных
conn.close()

(1, '16', 'Декабря', 'Вектор — на устойчивое развитие: «Ростелеком» закрепил лидирующие позиции в рейтинге РСПП', 'https://www.company.rt.ru/ir/news_calendar/d472031/')
(2, '13', 'Ноября', 'Дмитрий Медведев провел заседание совета директоров «Ростелекома»', 'https://www.company.rt.ru/ir/news_calendar/d471817/')
(3, '12', 'Ноября', 'Финансовые и операционные результаты деятельности «Ростелекома» за III\xa0квартал и 9\xa0месяцев 2024\xa0г.', 'https://www.company.rt.ru/ir/news_calendar/d471790/')
(4, '30', 'Октября', 'AK&M подтвердило высший рейтинг нефинансовой отчетности «Ростелекома»', 'https://www.company.rt.ru/ir/news_calendar/d471707/')
(5, '02', 'Октября', 'Дмитрий Медведев избран председателем Совета директоров «Ростелекома»', 'https://www.company.rt.ru/ir/news_calendar/d471524/')
(6, '13', 'Ноября', '«Ростелеком» разместил биржевые облигации на сумму 20 млрд рублей, ставка купона составит 23% годовых', 'https://www.company.rt.ru/ir/news_calendar/d471814/')
(7, '16', 'Сентября', '

In [11]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/news')
def news():
    conn = sqlite3.connect('news.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM news")
    rows = cursor.fetchall()

    conn.close()
    
    return render_template('news.html', gifts=rows)

if __name__ == '__main__':
    app.run(host='0.0.0.0')

#debug=True,

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.88.250:5000
Press CTRL+C to quit
192.168.88.250 - - [20/Dec/2024 07:49:23] "GET / HTTP/1.1" 200 -
192.168.88.250 - - [20/Dec/2024 07:50:26] "GET /news HTTP/1.1" 200 -
